In [38]:
!pip install sexpdata

In [42]:
import subprocess, sexpdata
from sexpdata import loads

proc = subprocess.Popen(
    ["sertop"],
    stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True
)

def read(_proc):
    line = "("
    while "Completed" not in line:
        line += _proc.stdout.readline()
        print(line.split("\n")[-2])
    return line + ")"

line = proc.stdout.readline()
while "(contents Processed)" not in line:
    line = proc.stdout.readline()


In [43]:
cmd = '(Add () "Lemma test: forall x: nat, x = x.")\n'
proc.stdin.write(cmd)
proc.stdin.flush()

In [44]:
read(proc)

((Answer 0 Ack)
(Answer 0(Added 2((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 33))NewAddTip))
(Answer 0 Completed)


'((Answer 0 Ack)\n(Answer 0(Added 2((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 33))NewAddTip))\n(Answer 0 Completed)\n)'

In [45]:
# Выполняем добавленный фрагмент с id = 2
cmd = '(Exec 2)\n'
proc.stdin.write(cmd)
proc.stdin.flush()

# Читаем ответы
read(proc)


((Answer 1 Ack)
(Feedback((doc_id 0)(span_id 2)(route 0)(contents(ProcessingIn master))))
(Feedback((doc_id 0)(span_id 1)(route 0)(contents Processed)))
(Feedback((doc_id 0)(span_id 2)(route 0)(contents Processed)))
(Answer 1 Completed)


'((Answer 1 Ack)\n(Feedback((doc_id 0)(span_id 2)(route 0)(contents(ProcessingIn master))))\n(Feedback((doc_id 0)(span_id 1)(route 0)(contents Processed)))\n(Feedback((doc_id 0)(span_id 2)(route 0)(contents Processed)))\n(Answer 1 Completed)\n)'

In [46]:
cmd = '(Add () "Proof.")\n'
proc.stdin.write(cmd)
proc.stdin.flush()

# Читаем ответы
read(proc)

cmd = '(Exec 3)\n'
proc.stdin.write(cmd)
proc.stdin.flush()

# Читаем ответы
read(proc)

((Answer 2 Ack)
(Answer 2(Added 3((fname ToplevelInput)(line_nb 1)(bol_pos 0)(line_nb_last 1)(bol_pos_last 0)(bp 0)(ep 6))NewAddTip))
(Answer 2 Completed)
((Answer 3 Ack)
(Feedback((doc_id 0)(span_id 3)(route 0)(contents(ProcessingIn master))))
(Feedback((doc_id 0)(span_id 2)(route 0)(contents Processed)))
(Feedback((doc_id 0)(span_id 3)(route 0)(contents Processed)))
(Answer 3 Completed)


'((Answer 3 Ack)\n(Feedback((doc_id 0)(span_id 3)(route 0)(contents(ProcessingIn master))))\n(Feedback((doc_id 0)(span_id 2)(route 0)(contents Processed)))\n(Feedback((doc_id 0)(span_id 3)(route 0)(contents Processed)))\n(Answer 3 Completed)\n)'

In [49]:
cmd = '(Query ((sid 3)) Goals)\n'
proc.stdin.write(cmd)
proc.stdin.flush()

# Читаем ответы
all_ans = loads(read(proc))

((Answer 5 Ack)
(Answer 5(ObjList((CoqGoal((goals(((info((evar(Ser_Evar 2))(name())))(ty(Prod((binder_name(Name(Id x)))(binder_relevance Relevant))(Ind(((MutInd(KerName(MPfile(DirPath((Id Datatypes)(Id Init)(Id Coq))))(Id nat))())0)(Instance(()()))))(App(Ind(((MutInd(KerName(MPfile(DirPath((Id Logic)(Id Init)(Id Coq))))(Id eq))())0)(Instance(()()))))((Ind(((MutInd(KerName(MPfile(DirPath((Id Datatypes)(Id Init)(Id Coq))))(Id nat))())0)(Instance(()()))))(Rel 1)(Rel 1)))))(hyp()))))(stack())(bullet())(shelf())(given_up()))))))
(Answer 5 Completed)


In [59]:
all_ans

[[Symbol('Answer'), 5, Symbol('Ack')],
 [Symbol('Answer'),
  5,
  [Symbol('ObjList'),
   [[Symbol('CoqGoal'),
     [[Symbol('goals'),
       [[[Symbol('info'),
          [[Symbol('evar'), [Symbol('Ser_Evar'), 2]], [Symbol('name'), []]]],
         [Symbol('ty'),
          [Symbol('Prod'),
           [[Symbol('binder_name'),
             [Symbol('Name'), [Symbol('Id'), Symbol('x')]]],
            [Symbol('binder_relevance'), Symbol('Relevant')]],
           [Symbol('Ind'),
            [[[Symbol('MutInd'),
               [Symbol('KerName'),
                [Symbol('MPfile'),
                 [Symbol('DirPath'),
                  [[Symbol('Id'), Symbol('Datatypes')],
                   [Symbol('Id'), Symbol('Init')],
                   [Symbol('Id'), Symbol('Coq')]]]],
                [Symbol('Id'), Symbol('nat')]],
               []],
              0],
             [Symbol('Instance'), [[], []]]]],
           [Symbol('App'),
            [Symbol('Ind'),
             [[[Symbol('MutInd'),
  

In [75]:
def bypass(_list, _print=False, _id=[]):
    _id.append("0")
    is_this_level = False
    for x in _list:
        if isinstance(x, sexpdata.Symbol):
            if x.value() == "Prod":
                _print = True
                is_this_level = True
        if isinstance(x, list):
            bypass(x, _print)
            if is_this_level:
                _print = False
        elif _print:
            print("[", ",".join(_id), "]", x)
        _id[-1] = str(int(_id[-1]) + 1)
    _id.pop()

In [76]:
bypass(all_ans)

[ 1,2,1,0,1,0,1,0,1,1,0 ] Prod
[ 1,2,1,0,1,0,1,0,1,1,1,0,0 ] binder_name
[ 1,2,1,0,1,0,1,0,1,1,1,0,1,0 ] Name
[ 1,2,1,0,1,0,1,0,1,1,1,0,1,1,0 ] Id
[ 1,2,1,0,1,0,1,0,1,1,1,0,1,1,1 ] x
[ 1,2,1,0,1,0,1,0,1,1,1,1,0 ] binder_relevance
[ 1,2,1,0,1,0,1,0,1,1,1,1,1 ] Relevant
